In [2]:
source extend_bashkernel.source

Ready.


In [ ]:
# (1) optional reset of Wakame-vdc
# Comment out the following lines and run cell to remove all
# added images and instances.

# reset-wakame.sh

In [3]:
# (2) Create Centos instance using the Wakame-vdc GUI

# If needed, one of these can
# be entered on the next line: (hint, restore, or check)
# restore

Now executing the following script to do this step:
cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

reset-wakame.sh

sleep 10

mussel instance create --cpu-cores 2 --hypervisor openvz     --image-id wmi-centos1d64 --memory-size 2048     --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

Output follows:
Terminating instance: i-43p7kbwc
---
- i-43p7kbwc
---
:id: i-n84n7t3q
:account_id: a-shpoolxx
:host_node: 
:cpu_cores: 2
:memory_size: 2048
:arch: x86_64
:image_id: wmi-centos1d64
:created_at: 2016-02-17 09:29:18.000000000 Z
:updated_at: 2016-02-17 09:29:18.509242323 Z
:terminated_at: 
:deleted_at: 
:state: scheduling
:status: init
:ssh_key_pair:
  :uuid: ssh-cicddemo
  :display_name: cicddemo
:volume:
- :vol_id: vol-989vmkr8
  :state: scheduling
:vif: []
:hostname: n84n7t3q
:ha_enabled: 0
:hypervisor: openvz
:display_name: centos
:service_type: std
:monitoring:
  :enabled: false
  :mail_address: []
  :items: {}
:labels:
- :reso

In [ ]:
# (3) Wait for instance's ssh port to become active
: ${IP:=10.0.2.100}
while [[ "$(echo | nc -w 1 "$IP" 22)" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port 22"

## repoファイルのインストール

In [5]:
ssh -qi ../mykeypair root@10.0.2.100 curl -fSkL http://pkg.jenkins-ci.org/redhat/jenkins.repo -o /etc/yum.repos.d/jenkins.repo

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    75    0    75    0     0     10      0 --:--:--  0:00:07 --:--:--    19


## 公開鍵のインポート

In [6]:
ssh -qi ../mykeypair root@10.0.2.100 rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key

## java実行環境のインストール

In [8]:
# This installs the Oracle version of Java.  The RPM file was
# already downloaded to /home/centos/notebooks/downloads

cd /home/centos/notebooks/downloads
tar c jdk-8u73-linux-x64.rpm | ssh -qi /home/centos/mykeypair root@10.0.2.100 tar xv

ssh -qi /home/centos/mykeypair root@10.0.2.100 <<EOS

rpm -ivh jdk-8u73-linux-x64.rpm

cat <<CFG >/etc/profile.d/java.sh
JAVA_HOME=/usr/java/jdk1.8.0_25/
PATH=$JAVA_HOME/bin:$PATH
export PATH JAVA_HOME
export CLASSPATH=.
CFG

EOS

jdk-8u73-linux-x64.rpm
Pseudo-terminal will not be allocated because stdin is not a terminal.
Preparing...                ##################################################
jdk1.8.0_73                 ##################################################
Unpacking JAR files...
	tools.jar...
	plugin.jar...
	javaws.jar...
	deploy.jar...
	rt.jar...
	jsse.jar...
	charsets.jar...
	localedata.jar...
	jfxrt.jar...


## jenkins coreのインストール

In [9]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y jenkins

Loaded plugins: fastestmirror
Setting up Install Process
Determining fastest mirrors
 * base: ftp.iij.ad.jp
 * extras: ftp.iij.ad.jp
 * updates: ftp.iij.ad.jp
Resolving Dependencies
--> Running transaction check
---> Package jenkins.noarch 0:1.647-1.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package           Arch             Version             Repository         Size
Installing:
 jenkins           noarch           1.647-1.1           jenkins            60 M

Transaction Summary
Install       1 Package(s)

Total download size: 60 M
Installed size: 60 M
Running rpm_check_debug
Running Transaction Test
Transaction Test Succeeded
Running Transaction
  Installing : jenkins-1.647-1.1.noarch                                     1/1 
  Verifying  : jenkins-1.647-1.1.noarch                                     1/1 

Installed:
  jenkins.noarch 0:1.647-1.1                                                    

Complete!


## 関連するビルドツールのインストール

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 yum install -y \
    git \
    iputils nc \
    qemu-kvm qemu-img \
    parted kpartx \
    rpm-build automake createrepo \
    openssl-devel zlib-devel readline-devel \
    gcc

Loaded plugins: fastestmirror
Setting up Install Process
Loading mirror speeds from cached hostfile
 * base: ftp.iij.ad.jp
 * extras: ftp.iij.ad.jp
 * updates: ftp.iij.ad.jp
Resolving Dependencies
--> Running transaction check
---> Package automake.noarch 0:1.11.1-4.el6 will be installed
--> Processing Dependency: perl >= 5.006 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.005_03 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.005 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl >= 5.000 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: autoconf >= 2.62 for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(vars) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(strict) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(constant) for package: automake-1.11.1-4.el6.noarch
--> Processing Dependency: perl(POSIX) for p

## sudo設定

### sudo権限の付与


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 "echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers"
ssh -qi ../mykeypair root@10.0.2.100 grep jenkins /etc/sudoers

### tty設定の変更

In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 'sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers'

### Loosen Java Security so Jenkins Plugins will work

In [ ]:
# see https://issues.jenkins-ci.org/browse/JENKINS-31089
javadir=/usr/lib/jvm/java-1.7.0-openjdk-1.7.0.95.x86_64/
secfile=jre/lib/security/java.security
ssh -qi ../mykeypair root@10.0.2.100 \
   "sudo sed -i 's/, MD5, RSA keySize < 1024//' $javadir/$secfile"


## jenkinsの起動


In [ ]:
ssh -qi ../mykeypair root@10.0.2.100 service jenkins start